# RUI Reporter - SenNet Analysis

This notebook analyzes RUI location registration coverage for SenNet datasets using the SenNet APIs.

## List of Supported Reference Organs

In [10]:
import requests

# HRA API endpoint for reference organs
REFERENCE_ORGANS_URL = "https://apps.humanatlas.io/api/v1/reference-organs"

# Get all reference organs
response = requests.get(REFERENCE_ORGANS_URL)
organs = response.json()

# Extract and normalize the UBERON IDs from the API
def iri_to_curie(iri):
    if iri and "obo/UBERON_" in iri:
        return "UBERON:" + iri.split("_")[-1]
    return iri

reference_uberon_ids = {iri_to_curie(organ.get("representation_of")) for organ in organs if organ.get("representation_of")}

print(f"{len(organs)} Supported Reference Organs")
print(f"Extracted {len(reference_uberon_ids)} unique UBERON IDs")

73 Supported Reference Organs
Extracted 42 unique UBERON IDs


## Ratio of Registered/Total for SenNet

In [11]:
import requests

# --- Setup reference organs ---
# HRA API endpoint for reference organs
REFERENCE_ORGANS_URL = "https://apps.humanatlas.io/api/v1/reference-organs"

# Get all reference organs
response = requests.get(REFERENCE_ORGANS_URL)
organs = response.json()

# Extract and normalize the UBERON IDs from the API
def iri_to_curie(iri):
    if iri and "obo/UBERON_" in iri:
        return "UBERON:" + iri.split("_")[-1]
    return iri

reference_uberon_ids = {iri_to_curie(organ.get("representation_of")) for organ in organs if organ.get("representation_of")}

# --- User-provided SenNet API token ---
SENNET_TOKEN = "AgKKovd81aw6X1W9z0O5G0mMXyzln3zWja6qGE5dEXQwxyBOoHeC4Xo2Jkg6jOBed6kXB2xOBEPWXUDa61qOhb52Y6"

# SenNet API endpoints
SEARCH_API_URL = "https://search.api.sennetconsortium.org/search"
headers = {"Authorization": f"Bearer {SENNET_TOKEN}", "Content-Type": "application/json"}

# Get total count of all datasets
total_query = {
    "version": True,
    "size": 0,
    "track_total_hits": True,
    "query": {
        "bool": {
            "filter": [
                {"term": {"entity_type.keyword": "Dataset"}},
                {"term": {"creation_action.keyword": "Create Dataset Activity"}}
            ]
        }
    }
}

total_response = requests.post(SEARCH_API_URL, json=total_query, headers=headers)
total_datasets_count = total_response.json()['hits']['total']['value'] if total_response.status_code == 200 else 0

# Get count of registered datasets (with RUI information)
registered_query = {
    "query": {
        "bool": {
            "filter": [
                {
                    "terms": {
                        "entity_type.keyword": [
                            "Dataset"
                        ]
                    }
                },
                {
                    "terms": {
                        "creation_action.keyword": [
                            "Create Dataset Activity"
                        ]
                    }
                },
                {
                    "terms": {
                        "has_rui_information.keyword": [
                            "True"
                        ]
                    }
                }
            ]
        }
    },
    "from": 0,
    "size": 0,
    "track_total_hits": True
}

registered_response = requests.post(SEARCH_API_URL, json=registered_query, headers=headers)
registered_count = registered_response.json()['hits']['total']['value'] if registered_response.status_code == 200 else 0

# Get count of supported datasets (with or without RUI information)
supported_query = {
    "query": {
        "bool": {
            "filter": [
                {
                    "terms": {
                        "entity_type.keyword": [
                            "Dataset"
                        ]
                    }
                },
                {
                    "terms": {
                        "creation_action.keyword": [
                            "Create Dataset Activity"
                        ]
                    }
                },
                {
                    "terms": {
                        "has_rui_information.keyword": [
                            "True",
                            "False"
                        ]
                    }
                }
            ]
        }
    },
    "from": 0,
    "size": 0,
    "track_total_hits": True
}

supported_response = requests.post(SEARCH_API_URL, json=supported_query, headers=headers)
supported_count = supported_response.json()['hits']['total']['value'] if supported_response.status_code == 200 else 0

print(f"Total datasets: {total_datasets_count}")
print(f"Supported datasets: {supported_count}")
print(f"Registered datasets: {registered_count}")

Total datasets: 2147
Supported datasets: 1504
Registered datasets: 1276
